In [12]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [13]:
class ConvRF(object):
    def __init__(self, kernel_size=5, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride
        self.kernel_forests = None
        self.num_outputs = 10

    def _convolve(self, images, labels=None, flatten=False):

        batch_size, in_dim, _, num_channels = images.shape

        out_dim = int((in_dim - self.kernel_size) / self.stride) + 1  # calculate output dimensions

        # create matrix to hold the chopped images
        out_images = np.zeros((batch_size, out_dim, out_dim,
                               self.kernel_size, self.kernel_size, num_channels))
        out_labels = None

        curr_y = out_y = 0
        # move kernel vertically across the image
        while curr_y + self.kernel_size <= in_dim:
            curr_x = out_x = 0
            # move kernel horizontally across the image
            while curr_x + self.kernel_size <= in_dim:
                # chop images
                out_images[:, out_x, out_y] = images[:, curr_x:curr_x +
                                                     self.kernel_size, curr_y:curr_y+self.kernel_size, :]
                curr_x += self.stride
                out_x += 1
            curr_y += self.stride
            out_y += 1

        if flatten:
            out_images = out_images.reshape(batch_size, out_dim, out_dim, -1)

        if labels is not None:
            out_labels = np.zeros((batch_size, out_dim, out_dim))
            out_labels[:, ] = labels.reshape(-1, 1, 1)

        return out_images, out_labels, out_dim

    def convolve_fit(self, images, labels):
        sub_images, sub_labels, out_dim = self._convolve(images, labels=labels, flatten=True)

        self.kernel_forests = [[0]*out_dim for _ in range(out_dim)]
        convolved_image = np.zeros((images.shape[0], out_dim, out_dim, self.num_outputs))
        for i in range(out_dim):
            for j in range(out_dim):
                self.kernel_forests[i][j] = RandomForestClassifier()
                self.kernel_forests[i][j].fit(sub_images[:, i, j], sub_labels[:, i, j])
#                 convolved_image[:, i, j] = self.kernel_forests[i][j].predict_proba(sub_images[:, i, j])
                convolved_image[:, i, j] = self.kernel_forests[i][j].apply(sub_images[:, i, j])

        return convolved_image

    def convolve_predict(self, images):
        if not self.kernel_forests:
            raise Exception("Should fit training data before predicting")

        sub_images, _, out_dim = self._convolve(images, flatten=True)

        kernel_predictions = np.zeros((images.shape[0], out_dim, out_dim, self.num_outputs))
        for i in range(out_dim):
            for j in range(out_dim):
#                 kernel_predictions[:, i, j] = self.kernel_forests[i][j].predict_proba(sub_images[:, i, j])
                kernel_predictions[:, i, j] = self.kernel_forests[i][j].apply(sub_images[:, i, j])

        return kernel_predictions

In [14]:
# prepare MNIST data
import torchvision.datasets as datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_train_images = mnist_trainset.train_data.numpy()[..., np.newaxis]
mnist_train_labels = mnist_trainset.train_labels.numpy()

# X_train, X_test, y_train, y_test = train_test_split(mnist_train_images, mnist_train_labels,
#                                                     stratify=mnist_train_labels,
#                                                     test_size=0.25)
# mnist_train_images = X_train
# mnist_train_labels = y_train
# mnist_test_images = X_test
# mnist_test_labels = y_test

mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)
mnist_test_images = mnist_testset.test_data.numpy()[..., np.newaxis]
mnist_test_labels = mnist_testset.test_labels.numpy()

In [15]:
def run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels):
    print("Num. of Convolution Layers: 1")
    # conv layer 1
    conv1 = ConvRF(kernel_size=3, stride=2)
    conv1_map = conv1.convolve_fit(mnist_train_images, mnist_train_labels)
    
    # full RF (conv 1)
    conv1_full_RF = RandomForestClassifier(n_estimators=100)
    conv1_full_RF.fit(conv1_map.reshape(len(mnist_train_images), -1), mnist_train_labels)
    
    # test (conv 1)
    conv1_map_test = conv1.convolve_predict(mnist_test_images)
    mnist_test_preds = conv1_full_RF.predict(conv1_map_test.reshape(len(mnist_test_images), -1))

    print("Test Accuracy: " + str(accuracy_score(mnist_test_labels, mnist_test_preds)))
    print("Validation Confusion Matrix: \n" + str(confusion_matrix(mnist_test_labels, mnist_test_preds)))
    
    print("Num. of Convolution Layers: 2")
    # conv layer 2
    conv2 = ConvRF(kernel_size=3, stride=2)
    conv2_map = conv2.convolve_fit(conv1_map, mnist_train_labels)
    
    # full RF (conv 2)
    conv2_full_RF = RandomForestClassifier(n_estimators=100)
    conv2_full_RF.fit(conv2_map.reshape(len(mnist_train_images), -1), mnist_train_labels)
    
    # test (conv 2)
    conv2_map_test = conv2.convolve_predict(conv1_map_test)
    mnist_test_preds = conv2_full_RF.predict(conv2_map_test.reshape(len(mnist_test_images), -1))

    print("Test Accuracy: " + str(accuracy_score(mnist_test_labels, mnist_test_preds)))
    print("Validation Confusion Matrix: \n" + str(confusion_matrix(mnist_test_labels, mnist_test_preds)))

    print("Num. of Convolution Layers: 3")
    # conv layer 3
    conv3 = ConvRF(kernel_size=3, stride=1)
    conv3_map = conv3.convolve_fit(conv2_map, mnist_train_labels)
    
    # full RF (conv 3)
    conv3_full_RF = RandomForestClassifier(n_estimators=100)
    conv3_full_RF.fit(conv3_map.reshape(len(mnist_train_images), -1), mnist_train_labels)
    
    # test (conv 3)
    conv3_map_test = conv3.convolve_predict(conv2_map_test)
    mnist_test_preds = conv3_full_RF.predict(conv3_map_test.reshape(len(mnist_test_images), -1))

    print("Test Accuracy: " + str(accuracy_score(mnist_test_labels, mnist_test_preds)))
    print("Validation Confusion Matrix: \n" + str(confusion_matrix(mnist_test_labels, mnist_test_preds)))

In [8]:
# when using rf.predict_proba()
run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels)

Num. of Convolution Layers: 1
Test Accuracy: 0.9289
Validation Confusion Matrix: 
[[ 950    0    7    3    1    3    8    4    3    1]
 [   0 1115    9    1    1    0    3    2    4    0]
 [  11    1  953    9    5    3    4   19   25    2]
 [   2    1   16  926    0   30    2    7   16   10]
 [   0    3    6    0  932    2    7    3    8   21]
 [   7    2    3   41    1  814    7    4   10    3]
 [  13    3    5    1    8   15  909    0    4    0]
 [   5   16   27    2   13    3    0  920    5   37]
 [  12    4   15   27    8   24    4    5  862   13]
 [  14   10    8   12   22    2    1   26    6  908]]
Num. of Convolution Layers: 2
Test Accuracy: 0.8494
Validation Confusion Matrix: 
[[ 861    0   10    8    1   15   58   20    4    3]
 [   0 1097   14    0    1    7    5    1   10    0]
 [  15   33  839   47   13   11   16   28   28    2]
 [   8    0   11  831    1  108    5   18   22    6]
 [   1   16   15    3  811   10   31   15    9   71]
 [   8    5    0  103    0  738   13    

In [11]:
# when using rf.apply()
run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels)

Num. of Convolution Layers: 1
Test Accuracy: 0.9685
Validation Confusion Matrix: 
[[ 970    1    0    0    1    4    1    1    1    1]
 [   0 1120    3    4    0    1    1    1    4    1]
 [   6    0 1000    3    5    0    1    9    8    0]
 [   0    1   12  969    0    6    0    9    8    5]
 [   1    0    0    0  956    0    5    0    5   15]
 [   4    1    0   13    3  852    8    1    5    5]
 [   4    2    1    0    2    6  938    0    4    1]
 [   2    4   11    1    2    0    0  992    5   11]
 [   5    0    4    7    6    5    6    3  928   10]
 [   5    4    3    7   14    2    1    4    9  960]]
Num. of Convolution Layers: 2
Test Accuracy: 0.9589
Validation Confusion Matrix: 
[[ 966    1    0    0    2    2    2    2    3    2]
 [   0 1116    2    5    1    1    4    1    5    0]
 [   4    0 1001    2    3    2    1    9   10    0]
 [   0    1   15  949    0   18    0    9   11    7]
 [   1    0    3    0  947    0    5    1    7   18]
 [   4    0    1   27    3  828   11    